In [3]:
import time
import json
import datetime
from math import ceil
from datetime import date, timedelta, datetime

import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd
import requests_cache
import requests as re

In [80]:
df = pd.read_csv('data/clusterization.csv')
df = df[['label', 'id']]
df['uris'] = 'spotify:track:' + df['id']

In [5]:
df.sample()

,label,id,uri
4685,3,7Fd9g5y4q8ApG79VQeLfop,spotify:track:7Fd9g5y4q8ApG79VQeLfop


#### [get your token here](https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=%7Bplaylist%7D&position=&uris=spotify%3Atrack%3A7uuWlqHI41LkdXn4pcqI1h%2Cspotify%3Atrack%3A15jdwHb5nOWJrxXvPavXTR%2Cspotify%3Atrack%3A2LwM4JgvJ3SiIaQOIXJT6n)

In [72]:
#authentication. for now it is a manyally requst token but I'll come back to that later
access_token = 'BQA0P4vTq7_VpbAZMdd_xwrRY1-4_WuaxWYMPYkEID-nNf-3Tu7Dw_zpwZJn-aLB3vyKOGUAxpZOfcGy95MBN3GksyXwo3Ro7aoxIA58TMrq04GQdqWSzFu2Lp7fv_uxjCDKgIY-bkjKhpztJyTYeLIEKo0Rjo8Jv8kQG_kC8CP-CSl7CXZt61eVK5fqkPTP8iFA2MBL'
headers = {"Content-Type":"application/json", "Authorization":"Bearer {token}".format(token=access_token)}
user_id = 'ommarra'

### Creating the playlists

**IMPORTANT:** the playlists should either be created at the same time or we'd need to manually create the playlists array with their ids
If needed, you can manually delete the playlists on Spotify and then re-create them

In [9]:
endpoint_url = "https://api.spotify.com/v1/users/{user_id}/playlists".format(user_id=user_id)

playlists = {}
for i in range(len(df['label'].unique())):
    label = df['label'].unique()[i].item()
    playlist_name = "k-means-cluster-0" + str(label)
    request_body = json.dumps({
            "name": playlist_name,
            "description": "k means generated. probably shitty",
            "public": True
            })
    r = re.post(url=endpoint_url, data=request_body, headers=headers)
    #generate a dict with the labels and the playlist id - this will be used to add the songs later
    playlists[label] = r.json()['id']

### Adding songs to the playlists

In [113]:
def add_songs_to_playlist(_songs, _playlist):
    playlist = _playlist
    url = "https://api.spotify.com/v1/playlists/{playlist}/tracks".format(playlist=playlist)
    data = json.dumps({ 'uris': _songs })
    response = re.post(url=url, data=data, headers=headers)
    print(response.json())

In [118]:
for key in playlists:
    playlist = playlists[key]
    #150 songs per playlist is a good starter. Later on I'd like to also limit the number of songs from the same artist
    tempdf = df[df['label'] == key].sample(150) 
    #break it even to post the ids in chunks
    splits = ceil(len(tempdf)/100)
    chunks = np.array_split(tempdf, splits)
    #format and call the function to every chunk
    for i in range(len(chunks)):
        songs = chunks[i]['uris'].to_json(orient='records')
        add_songs_to_playlist(_songs=json.loads(songs), _playlist=playlist)        

{'snapshot_id': 'MywwNjY0NjkxMTk4M2E2NmFhMmM1NzE2YzE3ZjM0NWM1OTdmYjllZjMw'}
{'snapshot_id': 'NCw1YmFmMTY0NDA4M2Y5MTY2ZDY0M2JlOWQzYWM5NjBkMzRlMGJkMzBi'}
{'snapshot_id': 'MywyOWJiYWYyMGQ3OTkyNDliOGFhMTE4MmVhYTdhNTViNjY1ZmJmZDMy'}
{'snapshot_id': 'NCxiYTJmOTVhOGJmYmQ4MTRjMTg1MDIzN2E3ZjYwZjlkYzJhZDY2MTcy'}
{'snapshot_id': 'Myw1Mjk5MWZmMjFmM2FiZTFkNjhkY2Y3ZWQyN2VmZjk4OGMyMDNlMTY2'}
{'snapshot_id': 'NCxmMjkzODQyNDFhMDU1NjI2NDA4ZGUyYjVjMTdiMmRmYmFlMzliM2Vi'}
{'snapshot_id': 'MyxjODFmZThkZTdmN2NlODA5YmQxYjA1NTc3M2RkNTZmY2EwYzRiNDcx'}
{'snapshot_id': 'NCw0Yzc0YzFhNDM2YWZhMDdjNDMzNjI0OGM2YmMxZTVmMDJiZmVmNmY5'}
